In [1]:
from params import *

In [2]:
rental_prices = pd.read_csv(f'{DATA_DIR}/rental-prices/rental-prices.csv')
wages = pd.read_csv(f'{DATA_DIR}/ashe/weekly-earnings.csv')

In [3]:
rental_prices = rental_prices[(rental_prices.date == max(rental_prices.date)) 
                              &(rental_prices.variable_code == 'Lower quartile')
                              &(rental_prices.property_name == 'bed2')]
rental_prices = rental_prices[['geography_code', 'value']].set_index('geography_code')
rental_prices.rename(columns={'value': 'lower_quartile_rent'}, inplace=True)
rental_prices

,lower_quartile_rent
geography_code,
E92000001,625
E12000001,430
E06000047,395
E06000005,425
E06000001,400
...,...
E10000027,630
E07000187,695
E07000188,575


In [4]:
wages = wages[
    (wages.date == max(wages.date)) 
    &(wages.variable_name == 'lq_monthly_wage')]
wages = wages[['geography_code', 'geography_name', 'value']].set_index('geography_code')
wages.rename(columns={'value': 'lq_monthly_wage'}, inplace=True)
wages

,geography_name,lq_monthly_wage
geography_code,,
E12000001,North East,1419.0
E06000005,Darlington UA,1580.0
E06000001,Hartlepool UA,1216.0
E06000002,Middlesbrough UA,NaN
E06000003,Redcar and Cleveland UA,1192.0
...,...,...
E08000032,Bradford,1435.0
E08000033,Calderdale,1567.0
E08000034,Kirklees,1428.0


In [5]:
merged = wages.merge(rental_prices, left_index=True, right_index=True)
print(merged.columns.to_list())
merged['lq_rent_to_lq_earnings'] = merged['lower_quartile_rent'] / merged['lq_monthly_wage']
merged

['geography_name', 'lq_monthly_wage', 'lower_quartile_rent']


,geography_name,lq_monthly_wage,lower_quartile_rent,lq_rent_to_lq_earnings
geography_code,,,,
E12000001,North East,1419.0,430,0.303030
E06000005,Darlington UA,1580.0,425,0.268987
E06000001,Hartlepool UA,1216.0,400,0.328947
E06000002,Middlesbrough UA,NaN,433,NaN
E06000003,Redcar and Cleveland UA,1192.0,425,0.356544
...,...,...,...,...
E08000032,Bradford,1435.0,475,0.331010
E08000033,Calderdale,1567.0,475,0.303127
E08000034,Kirklees,1428.0,470,0.329132


In [6]:
merged = merged.melt(value_vars='lq_rent_to_lq_earnings', 
                    id_vars='geography_name', 
                    ignore_index=False, 
                    var_name='variable_name'
                    )
merged = merged.dropna()
merged.to_csv(f'{DATA_DIR}/ashe/rent-earning-ratio.csv')